# Reading and Writing Timeseries from FITS Files
Here we're going to test how `yt` reads fits files and how easy it is to read/write time series information to and from FITS files.

In [10]:
import os
import sys

import sunpy.map
from sunpy.net import vso
from astropy.io import fits
import astropy.units as u
import numpy as np
import yt

%matplotlib inline

First, grab an SDO image so that we have some sample header file information.

In [8]:
client = vso.VSOClient()
result_aia = client.query(
    vso.attrs.Time((2013, 1, 1, 7, 34, 0), (2013, 1, 1, 9, 0, 0)), # Time range.
    vso.attrs.Instrument('AIA'),
    vso.attrs.Physobs('intensity'), 
    vso.attrs.Sample(5000 * u.s)    
)
data_aia = client.get(result_aia, methods=('URL-FILE_Rice', 'URL-FILE')).wait()

Exception ignored in: <bound method FITS_rec.__del__ of FITS_rec([(array([1698,    0], dtype=int32),),
       (array([1691, 1698], dtype=int32),),
       (array([1659, 3389], dtype=int32),), ...,
       (array([   1678, 7392734], dtype=int32),),
       (array([   1678, 7394412], dtype=int32),),
       (array([   1688, 7396090], dtype=int32),)], 
      dtype=(numpy.record, [('COMPRESSED_DATA', '>i4', (2,))]))>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.4/site-packages/astropy/io/fits/fitsrec.py", line 640, in __del__
    del self._coldefs
  File "/opt/anaconda3/lib/python3.4/site-packages/astropy/io/fits/fitsrec.py", line 637, in _coldefs
    raise AttributeError(exc.args[0])
AttributeError: _coldefs


In [165]:
aia_map = sunpy.map.Map(data_aia[0])

Exception ignored in: <bound method FITS_rec.__del__ of FITS_rec([(array([1698,    0], dtype=int32),),
       (array([1691, 1698], dtype=int32),),
       (array([1659, 3389], dtype=int32),), ...,
       (array([   1678, 7392734], dtype=int32),),
       (array([   1678, 7394412], dtype=int32),),
       (array([   1688, 7396090], dtype=int32),)], 
      dtype=(numpy.record, [('COMPRESSED_DATA', '>i4', (2,))]))>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.4/site-packages/astropy/io/fits/fitsrec.py", line 640, in __del__
    del self._coldefs
  File "/opt/anaconda3/lib/python3.4/site-packages/astropy/io/fits/fitsrec.py", line 637, in _coldefs
    raise AttributeError(exc.args[0])
AttributeError: _coldefs


Configure a default header. May need some work once we are doing this for real.

In [178]:
template = aia_map.wcs.to_header()

Write a series of FITS files so that they can be loaded as a timeseries into `yt`.

In [179]:
time = np.linspace(0.0,100.0,10)
template['bunit'] = 'cts/pixel/s*cm'
for t in time:
    #set some faux emissivity arrays
    a1 = np.random.rand(30,40,50)
    a2 = np.random.rand(30,40,50)
    a3 = np.random.rand(30,40,50) 
    #configure fits   
    template['time'] = t
    #template['NAXIS1'],template['NAXIS2'],template['NAXIS3'] = np.shape(a1)
    template['btype'] = 'A1'
    hdu1 = fits.PrimaryHDU(a1,header=template)
    #template['NAXIS1'],template['NAXIS2'],template['NAXIS3'] = np.shape(a2)
    template['btype'] = 'A2'
    hdu2 = fits.ImageHDU(a2,header=template)
    #template['NAXIS1'],template['NAXIS2'],template['NAXIS3'] = np.shape(a3)
    template['btype'] = 'A3'
    hdu3 = fits.ImageHDU(a3,header=template)
    #write fits file
    hdulist = fits.HDUList([hdu1,hdu2,hdu3])
    hdulist.writeto('/data/datadrive1/sample_yt_data/example_fits_tests/fits_test_{}.fits'.format(list(time).index(t)))

In [180]:
ts = yt.load('/data/datadrive1/sample_yt_data/example_fits_tests/fits_test_*.fits')

Have to set the time manually though it can be stored in the header so not the end of the world.

In [188]:
for ds in ts:
    ds.current_time = ds.primary_header['time']

yt : [WARNING  ] 2016-10-20 13:18:39,754 Cannot find time
yt : [WARNING  ] 2016-10-20 13:18:39,759 No length conversion provided. Assuming 1 = 1 cm.
yt : [INFO     ] 2016-10-20 13:18:39,763 Parameters: current_time              = 0.0
yt : [INFO     ] 2016-10-20 13:18:39,764 Parameters: domain_dimensions         = [50 40 30]
yt : [INFO     ] 2016-10-20 13:18:39,765 Parameters: domain_left_edge          = [ 0.5  0.5  0.5]
yt : [INFO     ] 2016-10-20 13:18:39,766 Parameters: domain_right_edge         = [ 50.5  40.5  30.5]
yt : [INFO     ] 2016-10-20 13:18:39,767 Parameters: cosmological_simulation   = 0.0
yt : [WARNING  ] 2016-10-20 13:18:39,781 Cannot find time
yt : [WARNING  ] 2016-10-20 13:18:39,786 No length conversion provided. Assuming 1 = 1 cm.
yt : [INFO     ] 2016-10-20 13:18:39,790 Parameters: current_time              = 0.0
yt : [INFO     ] 2016-10-20 13:18:39,791 Parameters: domain_dimensions         = [50 40 30]
yt : [INFO     ] 2016-10-20 13:18:39,792 Parameters: domain_left

In [189]:
#yt.SlicePlot(ts[9],'z','A2')
yt.PhasePlot(ts[9].all_data(),'A1','A2','A3')

yt : [WARNING  ] 2016-10-20 13:18:47,726 Cannot find time
yt : [WARNING  ] 2016-10-20 13:18:47,733 No length conversion provided. Assuming 1 = 1 cm.
yt : [INFO     ] 2016-10-20 13:18:47,739 Parameters: current_time              = 0.0
yt : [INFO     ] 2016-10-20 13:18:47,740 Parameters: domain_dimensions         = [50 40 30]
yt : [INFO     ] 2016-10-20 13:18:47,741 Parameters: domain_left_edge          = [ 0.5  0.5  0.5]
yt : [INFO     ] 2016-10-20 13:18:47,743 Parameters: domain_right_edge         = [ 50.5  40.5  30.5]
yt : [INFO     ] 2016-10-20 13:18:47,744 Parameters: cosmological_simulation   = 0.0


ReferenceError: weakly-referenced object no longer exists